In [1]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from Scripts.evaluation import evaluation_methods

In [5]:
data = np.load("E:\\@IIT_BBS\\@Sem 1\\ML\\Final Project\\Safety-Analysis\\Features\\Image Features\\combined_features.npz")
X = data["X"]
y = data["y"]


In [6]:

def custom_split(X, y):
    # First split: test = 100 samples
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y, test_size=116, random_state=42, shuffle=True
    )

    # Second split: validation = 100 samples
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val, test_size=0.5, random_state=42, shuffle=True
    )

    return X_train, X_val, X_test, y_train, y_val, y_test


X_train, X_val, X_test, y_train, y_val, y_test = custom_split(X, y)
print("Train shape:", X_train.shape, y_train.shape)
print("Validation shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

Train shape: (398, 817) (398,)
Validation shape: (398, 817) (398,)
Test shape: (116, 817) (116,)


In [7]:
from xgboost import XGBClassifier

# Create the model
model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.8,
    eval_metric='mlogloss',
    objective='multi:softprob',  # important for multi-class
    num_class=3,                 # put correct number of classes
    random_state=42
)



In [8]:
# Train the model
model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],  # validation monitoring
    verbose=True
)


[0]	validation_0-mlogloss:1.00887
[1]	validation_0-mlogloss:0.97848
[2]	validation_0-mlogloss:0.94847
[3]	validation_0-mlogloss:0.92003
[4]	validation_0-mlogloss:0.89515
[5]	validation_0-mlogloss:0.87222
[6]	validation_0-mlogloss:0.84786
[7]	validation_0-mlogloss:0.83007
[8]	validation_0-mlogloss:0.81351
[9]	validation_0-mlogloss:0.79505
[10]	validation_0-mlogloss:0.77981
[11]	validation_0-mlogloss:0.76293
[12]	validation_0-mlogloss:0.74696
[13]	validation_0-mlogloss:0.73386
[14]	validation_0-mlogloss:0.72219
[15]	validation_0-mlogloss:0.70944
[16]	validation_0-mlogloss:0.69705
[17]	validation_0-mlogloss:0.68596
[18]	validation_0-mlogloss:0.67657
[19]	validation_0-mlogloss:0.66514
[20]	validation_0-mlogloss:0.65653
[21]	validation_0-mlogloss:0.64663
[22]	validation_0-mlogloss:0.63850
[23]	validation_0-mlogloss:0.63122
[24]	validation_0-mlogloss:0.62408
[25]	validation_0-mlogloss:0.61728
[26]	validation_0-mlogloss:0.61095
[27]	validation_0-mlogloss:0.60398
[28]	validation_0-mlogloss:0.5

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None, num_class=3, ...)

In [13]:

# 3. Predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)



In [15]:
y_prob.shape

(116, 3)

In [16]:
# 4. Evaluate
eval_obj = evaluation_methods(y_test, y_pred, y_prob)
metrics_df = eval_obj.compute_metrics()
cm_file = eval_obj.save_confusion_matrix()

print(metrics_df)
print("Confusion matrix saved as:", cm_file)

# Save metrics to CSV
metrics_df.to_csv("evaluation_metrics.csv", index=False)

# Save ROC curve
roc_file = eval_obj.plot_roc_curve()
print("ROC curve saved as:", roc_file)

   Accuracy  Precision    Recall  F1 Score   ROC-AUC  Log Loss
0  0.793103   0.788629  0.793103  0.779775  0.929347  0.565684
Confusion matrix saved as: confusion_matrix.png
ROC curve saved as: roc_curve.png
